# Detect Anomalies Using Density Based Clustering

In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import itertools

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler


from env import get_connection

### Clustering - DBSCAN

- Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.  

In [2]:
url = get_connection("curriculum_logs")

sql = """
select *
from logs
"""

df = pd.read_sql(sql, url)
df.head()

,date,time,path,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [3]:
df.date = pd.to_datetime(df['date'])

In [4]:
df = df.set_index(df['date'])

In [5]:
df.head()

,date,time,path,user_id,cohort_id,ip
date,,,,,,
2018-01-26,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
2018-01-26,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


#### Selecting variables to cluster

In [6]:
# Cluster on all or a selected subset of features
cohort = df[['user_id', 'cohort_id']]
cohort.head()

,user_id,cohort_id
date,,
2018-01-26,1,8.0
2018-01-26,1,8.0
2018-01-26,1,8.0
2018-01-26,1,8.0
2018-01-26,2,22.0


In [7]:
cohort.iloc[0]

user_id      1.0
cohort_id    8.0
Name: 2018-01-26 00:00:00, dtype: float64

In [8]:
cohort.values.astype("float32", copy=False)[0]

array([1., 8.], dtype=float32)

In [9]:
np_array = cohort.values.astype('float32', copy=False)

### Scaling

In [10]:
# MAKE the scaler
mms = MinMaxScaler()

# FIT/transform the scaler
c_scaled = mms.fit_transform(np_array)

c_scaled

array([[0.        , 0.05072464],
       [0.        , 0.05072464],
       [0.        , 0.05072464],
       ...,
       [0.06428571, 0.19565219],
       [0.7581632 , 0.19565219],
       [0.06428571, 0.19565219]], dtype=float32)

In [11]:
scaled_columns = ["Scaled_"+column for column in cohort.columns]

In [12]:
# Save a copy of the original dataframe
original_df = df.copy()

In [13]:
# Create a dataframe containing the scaled array of values
scaled_df = pd.DataFrame(c_scaled, columns=scaled_columns)

In [14]:
scaled_df.head()

,Scaled_user_id,Scaled_cohort_id
0,0.00000,0.050725
1,0.00000,0.050725
2,0.00000,0.050725
3,0.00000,0.050725
4,0.00102,0.152174


In [ ]:
df.merge(scaled_df, on=df.index)

In [ ]:
df.head()

 
Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).